In [ ]:
#V2 will check if the path is empty in the csv. 
#If empty and available fill it,not empty not rewrite it
#v3 do it withoyt RT STRuctBuilder
#v4 again with RTStruct Builder,  because v3 was incomplete, but now it does it with both ITV and GTVs labels separately
#v5, same as v4 but clean

In [5]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import csv
import re
import pandas as pd

In [6]:
def LookForLabelMatch(ListOfNamesinRT,LabelsToLookFor,currPathTemp):
    tempList = []
    for labeltot in LabelsToLookFor:
        if labeltot in ListOfNamesinRT:
            tempList.append(currPathTemp)
            return tempList
    return tempList

In [7]:
def GetTheContourRTMatch(ctfolder,rtfolder,itv_tot_labels,itv_tumor_labels,itv_ln_labels,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels):
    singlematch = False # If one RT file has already been matched you dont need to loop to the next one. In theory only one RTStruct should be necessary
    ct_Match = []
    totGTV_Match, tumorGTV_Match, lnGTV_Match = ([],[],[])
    totITV_Match, tumorITV_Match, lnITV_Match = ([],[],[])
    gtv_bool = False
    itv_bool = False
    if len(ctfolder)==0:
        print("No CT")
    for k in range(len(ctfolder)):
        for j in range(len(rtfolder)):
            rtfiles = os.listdir(rtfolder[j])
            for i in range(len(rtfiles)):
                if not singlematch:
                    try:
                        print("check",i,j,k)
                        
                        rtstruct = RTStructBuilder.create_from(dicom_series_path=ctfolder[k],rt_struct_path=str(rtfolder[j]+rtfiles[i]))

                        totITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_tot_labels,str(rtfolder[j]+rtfiles[i]))
                        tumorITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_tumor_labels,str(rtfolder[j]+rtfiles[i]))
                        lnITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_ln_labels,str(rtfolder[j]+rtfiles[i]))

                        totGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_tot_labels,str(rtfolder[j]+rtfiles[i]))
                        tumorGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_tumor_labels,str(rtfolder[j]+rtfiles[i]))
                        lnGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_ln_labels,str(rtfolder[j]+rtfiles[i]))

                        if len(totITV_Match) ==0 and len(tumorITV_Match)+len(lnITV_Match) == 0:
                            print("No labels Matched for ITV",rtfiles[i],rtstruct.get_roi_names())
                        else: itv_bool = True
                        
                        if len(totGTV_Match) ==0 and len(tumorGTV_Match)+len(lnGTV_Match) == 0:
                            print("No labels Matched for GTV",rtfiles[i],rtstruct.get_roi_names())
                        else: gtv_bool = True
                        
                        if (len(totITV_Match)>0 or len(tumorITV_Match)+len(lnITV_Match)>0) or (len(totGTV_Match)>0 or len(tumorGTV_Match)+len(lnGTV_Match) >0):
                            ct_Match.append(ctfolder[k])
                            print("M ITV","tot",len(totITV_Match),"tumor",len(tumorITV_Match),"LN",len(lnITV_Match))
                            print("M GTV","tot",len(totGTV_Match),"tumor",len(tumorGTV_Match),"LN",len(lnGTV_Match))
                            if len(totITV_Match) == 0: totITV_Match.append('0')
                            if len(tumorITV_Match) == 0: tumorITV_Match.append('0')
                            if len(lnITV_Match) == 0: lnITV_Match.append('0')
                            if len(totGTV_Match) == 0: totGTV_Match.append('0')
                            if len(tumorGTV_Match) == 0: tumorGTV_Match.append('0')
                            if len(lnGTV_Match) == 0: lnGTV_Match.append('0')
                                
                        if gtv_bool and itv_bool:
                            singlematch = True
                    except Exception as e: print(e)#pass

        #mask_3d = rtstruct.get_roi_mask_by_name(label)
    return ct_Match,totITV_Match, tumorITV_Match, lnITV_Match,totGTV_Match, tumorGTV_Match, lnGTV_Match

In [8]:
#Hard coding labels, dont change
# ITV Labels
itv_tot_labels = ["ITV", "IGTV", "IgTV", "ITV1", "ITV2", "ITV3", "ITV totaal def", "2ITV", "ITV_TOT", "ITV_6000", "ITV_5100", "ITV_Totaal", "ITV_LBK", "ITV_LOK", "IGTV_6000"]
itv_tumor_labels = ["ITVtumor", "ITV_tumor", "ITVtumor def", "2ITV_tumor", "ITV-P", "ITVtumorA1", "ITV_tumor_LBK", "ITVtu", "IGTVp"]
itv_ln_labels = ["ITVklieren", "ITV_klier", "ITV_Klier", "ITVklieren def", "2ITV_klier", "ITV_n", "ITV_klier_LBK", "IGTVnode"]

# GTV Labels
gtv_tot_labels = ["GTV", "GTV1", "GTV2", "GTV3", "GTV totaal def", "2GTV", "GTV_TOT", "GTV_6000", "GTV_5100", "GTV_Totaal", "GTV_LBK", "GTV_LOK", "GTV_5000", "GTV_preop", "GTV_totaal", "GTV_voor OK", "GTVtotaal", "GTVnew", "GTV(op exp)", "GTVop", "GTV_voor OK", "GTV oorspr"]
gtv_tumor_labels = ["GTVtumor", "GTV_tumor", "GTVtumor def", "2GTV_tumor", "GTV-P", "GTVtumorA1", "GTV_tumor_LBK", "GTV tumor", "GTVp", "GTVtu", "GTVtumor_exp", "GTVtumor_exp_50->0", "GTVtumor_insp", "GTVpatelectasebewaren(1)", "GTVt"]
gtv_ln_labels = ["GTVklieren", "GTV_klier", "GTV_Klier", "GTVklieren def", "2GTV_klier", "GTV_n", "GTV_klier_LBK", "GTVn", "GTVnode_exp", "GTVnodes_exp", "GTVnodes_exp_50->0", "GTVnodes_insp", "GTVklieren"]

fieldnames=['ID', 'CT', 'ITVTot', 'ITVTumor', 'ITVLN', 'GTVtot', 'GTVTumor', 'GTVLN']


In [9]:
#GET PATHS AND LABELS
#Where to look for file
root_path  = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/"

#Where the results will be saved (CSV) [@Robert->Change this so it saves in your folder]
csv_file_path = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/CTandRTMatch_GTVandITV.csv"

#Patient list  to look for in txt [@Robert->Change this so it points to the list of patients you care about]
PxListPath = '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/MissingCTRTPx_Mar28.txt'
PxList = pd.read_csv(PxListPath,header=None,dtype=str)


In [10]:
#Look for patients already been analized (Ignore if this is the first time you run it)
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    checked_list = df['ID'].tolist()
    print(len(checked_list),"Px already checked,",len(PxList),"Missing")
else:
    checked_list = []

1032 Px already checked, 295 Missing


In [13]:
#MAIN 
with open(csv_file_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
        writer.writeheader()


    for Px in PxList[0]:
        strPx = str(Px)
        strPx = strPx.zfill(7)
        Px = strPx
        AvgCt_fpaths = []
        RTStruct_fpaths = []

        if False and int(Px) in checked_list:
            print(Px,"Px already checked")
        else:
            for root, dirs, files in os.walk(root_path+Px, topdown=True):
                for d in dirs:
                    if "Thorax" in d or "ave" in d.lower() or "MIP" in d: #("CT" in d and not "RTSTRUCT" in d): #"%" in d or 
                        AvgCt_fpaths.append(root+'/'+d+'/')
                    if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                        RTStruct_fpaths.append(root+'/'+d+'/')


            print("Px",Px,"Num CTs",len(AvgCt_fpaths),"Num RTs",len(RTStruct_fpaths))

            ctMatch,rtMatch_ITVtot,rtMatch_ITVtumor,rtMatch_ITVln,rtMatch_GTVtot,rtMatch_GTVtumor,rtMatch_GTVln = GetTheContourRTMatch(AvgCt_fpaths,RTStruct_fpaths,itv_tot_labels,itv_tumor_labels,itv_ln_labels,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels)

            if len(ctMatch) == 0 :
                print("No Match")
            else: 
                print("Yes Match")
                
                writer.writerow({'ID':Px,'CT':ctMatch,'ITVTot':rtMatch_ITVtot,'ITVTumor':rtMatch_ITVtumor,'ITVLN':rtMatch_ITVln,'GTVtot':rtMatch_GTVtot,'GTVTumor':rtMatch_GTVtumor,'GTVLN':rtMatch_GTVln})


Px 4291861 Num CTs 3 Num RTs 2
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000015102207115118000004805
check 0 1 0
Please check that the existing RTStruct is valid
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000015102207115118000004805
check 0 1 1
Please check that the existing RTStruct is valid
check 0 0 2
M ITV tot 1 tumor 0 LN 0
M GTV tot 1 tumor 0 LN 0
Yes Match
Px 3610529 Num CTs 2 Num RTs 1
check 0 0 0
No labels Matched for ITV IM1 (1).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT', 'Isoc N1F']
No labels Matched for GTV IM1 (1).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT', 'Isoc N1F']
check 1 0 0
Please check that the existing RTStruct is valid
check 2 0 0
No labels Matched for ITV IM1 (3).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT'

Please check that the existing RTStruct is valid
check 0 0 7
No labels Matched for ITV 2.16.840.1.113669.2.931128.676008997.20130917122725.662153.DCM ['IsocCT', 'IsocA1', 'A1d1', 'external', 'catheterpoetser', 'myelum', 'longli', 'longre', 'oesophagus', 'hart', 'itv60', 'ROI_1', 'ctv60', 'ptv60', 'hulp', 'long-itv']
No labels Matched for GTV 2.16.840.1.113669.2.931128.676008997.20130917122725.662153.DCM ['IsocCT', 'IsocA1', 'A1d1', 'external', 'catheterpoetser', 'myelum', 'longli', 'longre', 'oesophagus', 'hart', 'itv60', 'ROI_1', 'ctv60', 'ptv60', 'hulp', 'long-itv']
check 0 1 7
Please check that the existing RTStruct is valid
check 0 2 7
Please check that the existing RTStruct is valid
check 0 3 7
No labels Matched for ITV 2.16.840.1.113669.2.931128.676008997.20130917122725.662153.DCM ['IsocCT', 'IsocA1', 'A1d1', 'external', 'catheterpoetser', 'myelum', 'longli', 'longre', 'oesophagus', 'hart', 'itv60', 'ROI_1', 'ctv60', 'ptv60', 'hulp', 'long-itv']
No labels Matched for GTV 2.16.840

Please check that the existing RTStruct is valid
check 0 2 1
Please check that the existing RTStruct is valid
check 0 3 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.946.1371209134.864.1
check 0 4 1
Please check that the existing RTStruct is valid
check 0 0 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.946.1371209134.864.1
check 0 1 2
Please check that the existing RTStruct is valid
check 0 2 2
Please check that the existing RTStruct is valid
check 0 3 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.946.1371209134.864.1
check 0 4 2
Please check that the existing RTStruct is valid
No Match
Px 8879132 Num CTs 1 Num RTs 3
check 0 0 0
Loaded RTStruct references image(s) 

No DICOM Images found in input path
check 0 0 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.49238.30000014071006170520300005449
check 0 0 3
M ITV tot 0 tumor 1 LN 0
M GTV tot 1 tumor 0 LN 1
Yes Match
Px 2122758 Num CTs 2 Num RTs 2
check 0 0 0
[Errno 2] No such file or directory: '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/2122758\\1.2.840.113564.9.1.2792465697.55.2.5007445010/PinnPlan = A1Long (Wed Oct 15 103812 2014) RTSTRUCT/2.16.840.1.113669.2.931128.289629209.20141015105247.579046.DCM'
check 0 1 0
[Errno 2] No such file or directory: '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/2122758\\1.2.840.113564.9.1.2792465697.55.2.5007445010/PinnPlan = A1Long (Wed Oct 15 103819 2014) RTPLAN/2.16.840.1.113669.2.931128.289629209.20141015105251.819809.DCM'
check 0 0 1
[Errno 2] No such file or directo

Px 0403161 Num CTs 2 Num RTs 2
check 0 0 0
M ITV tot 1 tumor 1 LN 1
M GTV tot 1 tumor 1 LN 1
Yes Match
Px 2723765 Num CTs 1 Num RTs 1
check 0 0 0
No labels Matched for ITV 2.16.840.1.113669.2.931128.289629209.20151016142432.13944.DCM ['IsocCT IsocA1', 'external', 'catheterpoetser', 'myelum', 'long_li', 'long_re', 'oesophagus', 'hart', 'GTV', 'PTV51', 'CTV51', 'longen-gtv', 'myelum+marge', 'schilPTV51', 'hart-PTV51', 'Oes-PTV51', 'Hart-PTV+5mm', 'Oes-PTV+5mm', 'PTVring_klein', 'PTVring_groot', 'PTV-ITV', 'PTV-Longen', 'PTVinLongen', 'Internal', 'Huid5mm', 'blok', 'ROI_1', 'schil_buiten_long']
M ITV tot 0 tumor 0 LN 0
M GTV tot 1 tumor 0 LN 0
Yes Match
Px 0228708 Num CTs 2 Num RTs 1
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000015100207173406500000478
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Ins

...

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023051906245520500002847
check 0 0 9
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023051906245520500002847
check 0 0 10
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023051906245520500002847
check 0 0 11
No labels Matched for ITV IM1.DCM ['External', 'BODY', 'Internal5', 'Breast_R', 'Atrium_L', 'Ventricle_R', 'Ventricle_L', 'Lung_R', 'Atrium_R', 'Heart', 'Lung_L', 'Breast_L', 'Esophagus', 'SpinalCord', 'CTV_6000', 'PTV_6000', 'CTV_xxxx (1)', 'ICTV', 'ring3mmPTV_6000', 'Lung', 'Air_in_Target', 'Heart_Out', 'Esophagus_Out', 'Breast_L_Out', 'Breast_R_Out', 'Lung_Out', 'SpinalCord_plus_5mm', 'Lung minus CTV', 'PTV in Lung', 'Skin5mm',

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023070506483792400000844
check 0 1 13
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023070506483792400000844
No Match
Px 2386974 Num CTs 14 Num RTs 1
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 3
Loaded RTStruct re

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.7.130005.30000023082215471876900000486
check 0 1 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.7.130005.30000023082215471876900000486
check 0 0 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.7.130005.30000023082215471876900000486
check 0 1 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.7.130005.30000023082215471876900000486
check 0 0 3
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.7.130005.30000023082215471876900000486
check 0 1 3
Loaded RTStruct references image(s) that are not contai

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023083107393893400007561
check 0 0 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023083107393893400007561
check 0 0 3
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023083107393893400007561
check 0 0 4
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023083107393893400007561
check 0 0 5
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023083107393893400007561
check 0 0 6
Loaded RTStruct references image(s) that are not contained i

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023090606393100400007216
Yes Match
